In [1]:
import numpy as np
import random
import math
import time
import os
import matplotlib.cm as cm
import pandas as pd
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import plotly.express as px

from tqdm.notebook import tqdm
from math import sqrt
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from icecream import ic



## CALCE Data explore

In [2]:
# Load data
Battery = np.load("datasets/CALCE/CALCE.npy", allow_pickle=True)
Battery = Battery.item()
Battery.keys()

dict_keys(['CS2_35', 'CS2_36', 'CS2_37', 'CS2_38'])

### Combine all cell capcity into one csv

In [7]:
df_all_cell = Battery["CS2_38"]
df_all_cell["capacity_CS2_38"] = df_all_cell["capacity"]
for k, v in Battery.items():
    if k == "CS2_38":
        continue
    df_all_cell[f"capacity_{k}"] = v["capacity"]
droped_cols = ["capacity", "SoH", "resistance", "CCCT", "CVCT"]
df_all_cell = df_all_cell.drop(droped_cols, axis=1)
df_all_cell.to_csv("datasets/CALCE/CALCE.csv", index=False)

In [4]:
df_all_cell

,cycle,capacity_CS2_35,capacity_CS2_36,capacity_CS2_37
0,1.0,1.126385,1.133807,1.124251
1,2.0,1.126160,1.133413,1.123968
2,3.0,1.125966,1.132829,1.123611
3,4.0,1.118508,1.127033,1.117115
4,5.0,1.117210,1.126712,1.116239
...,...,...,...,...
991,992.0,NaN,NaN,NaN
992,993.0,NaN,NaN,NaN
993,994.0,NaN,NaN,NaN
994,995.0,NaN,NaN,NaN


In [5]:
# Create interactive line plot
fig = px.line(df_all_cell, x="cycle", y=["capacity_CS2_35", "capacity_CS2_36", "capacity_CS2_37", "capacity_CS2_38"],
              labels={"cycle": "Cycle Number", "value": "Capacity"},
              title="Battery Capacity Over Cycles")

# Show plot
fig.show()

ValueError: All arguments should have the same length. The length of argument `y` is 4, whereas the length of previously-processed arguments ['cycle'] is 996

### Prediction 1: assume next cycle capcit equals to current one

In [ ]:
# use CS2_35
RE_list_raw = list(df_all_cell["capacity_CS2_35"].diff(1)/df_all_cell["capacity_CS2_35"])[1:]
RE_list = [abs(i) for i in RE_list_raw]
RE_mean = sum(RE_list)/len(RE_list)
RE_mean

0.00716777761082984

### Prediction 1 extension: give a constant decrease for each cycle 0

In [ ]:
def calculate_re_with_diff(df, diff):
    diff_capacities = list(df["capacity_CS2_35"].diff(1)/df["capacity_CS2_35"])[1:]
    res = sum([abs(i + diff) for i in diff_capacities])/len(diff_capacities)
    return res

In [ ]:
x = [i * 0.0001 for i in range(1, 100,1)]
y = [calculate_re_with_diff(df_all_cell, i) for i in x]
fig = px.line(x=x, y=y)
fig.show()